In [25]:
import numpy as np
import pandas as pd


from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, GRU, LSTM

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [26]:
df = pd.read_csv(
"/kaggle/input/manipal-weather/manipal_atmospherics_df.csv",
sep=",",
engine="python",
skiprows=4
)


df.head()

,2011-01-04,24.7,0.00,19.69,27.7,17.4,4.09,36798.94,325,1009.7,70,20.4,0.278
0,2011-01-05,25.4,0.0,17.77,28.5,15.9,3.91,36494.80,309,1008.7,85,20.4,0.276
1,2011-01-06,26.2,0.0,19.31,28.8,18.4,4.45,36924.74,319,1008.7,61,19.4,0.273
2,2011-01-07,25.6,0.0,20.26,28.0,15.6,4.52,37417.72,178,1008.6,12,18.6,0.270
3,2011-01-08,25.2,0.0,20.29,28.2,15.1,4.38,37473.21,313,1008.3,29,20.0,0.268
4,2011-01-09,25.2,0.0,20.82,28.4,18.3,4.49,37701.15,297,1008.0,52,20.0,0.266


In [27]:
import numpy as np
import pandas as pd

df = pd.read_csv(
    "/kaggle/input/manipal-weather/manipal_atmospherics_df.csv",
    sep=",",
    engine="python",
    skiprows=3
)

df.head()


,time,temperature_2m_mean (°C),precipitation_sum (mm),shortwave_radiation_sum (MJ/m²),apparent_temperature_mean (°C),wind_speed_10m_max (km/h),et0_fao_evapotranspiration (mm),sunshine_duration (s),wind_direction_10m_dominant (°),pressure_msl_mean (hPa),cloud_cover_mean (%),dew_point_2m_mean (°C),soil_moisture_0_to_7cm_mean (m³/m³)
0,2011-01-04,24.7,0.0,19.69,27.7,17.4,4.09,36798.94,325,1009.7,70,20.4,0.278
1,2011-01-05,25.4,0.0,17.77,28.5,15.9,3.91,36494.80,309,1008.7,85,20.4,0.276
2,2011-01-06,26.2,0.0,19.31,28.8,18.4,4.45,36924.74,319,1008.7,61,19.4,0.273
3,2011-01-07,25.6,0.0,20.26,28.0,15.6,4.52,37417.72,178,1008.6,12,18.6,0.270
4,2011-01-08,25.2,0.0,20.29,28.2,15.1,4.38,37473.21,313,1008.3,29,20.0,0.268


In [28]:
df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')

df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5480 entries, 2011-01-04 to 2026-01-04
Data columns (total 12 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   temperature_2m_mean (°C)             5480 non-null   float64
 1   precipitation_sum (mm)               5480 non-null   float64
 2   shortwave_radiation_sum (MJ/m²)      5480 non-null   float64
 3   apparent_temperature_mean (°C)       5480 non-null   float64
 4   wind_speed_10m_max (km/h)            5480 non-null   float64
 5   et0_fao_evapotranspiration (mm)      5480 non-null   float64
 6   sunshine_duration (s)                5480 non-null   float64
 7   wind_direction_10m_dominant (°)      5480 non-null   int64  
 8   pressure_msl_mean (hPa)              5480 non-null   float64
 9   cloud_cover_mean (%)                 5480 non-null   int64  
 10  dew_point_2m_mean (°C)               5480 non-null   float64
 11  soil_moistur

In [29]:
df.columns = df.columns.str.strip()


df = df.rename(columns={
    'temperature_2m_mean (°C)': 'temperature'
})


In [30]:
train_df = df.loc[:'2024-12-31']
test_df = df.loc['2025-01-01':]


train_df.shape, test_df.shape

((5111, 12), (369, 12))

In [31]:
scaler = StandardScaler()


train_scaled = scaler.fit_transform(train_df[['temperature']])
test_scaled = scaler.transform(test_df[['temperature']])

In [32]:
WINDOW = 30


def create_sequences(data, window):
    X, y = [], []
    for i in range(len(data) - window):
        X.append(data[i:i+window])
        y.append(data[i+window])
    return np.array(X), np.array(y)


X_train, y_train = create_sequences(train_scaled, WINDOW)
X_test, y_test = create_sequences(test_scaled, WINDOW)


X_train.shape, X_test.shape

((5081, 30, 1), (339, 30, 1))

In [33]:
def build_rnn():
    model = Sequential([
        SimpleRNN(32, input_shape=(WINDOW, 1)),
        Dense(1)
])
    model.compile(optimizer='adam', loss='mse')
    return model




def build_gru():
    model = Sequential([
        GRU(32, input_shape=(WINDOW, 1)),
    Dense(1)
])
    model.compile(optimizer='adam', loss='mse')
    return model




def build_lstm():
    model = Sequential([
        LSTM(32, input_shape=(WINDOW, 1)),
    Dense(1)
])
    model.compile(optimizer='adam', loss='mse')
    return model

In [34]:
EPOCHS = 100


rnn = build_rnn()
gru = build_gru()
lstm = build_lstm()


rnn.fit(X_train, y_train, epochs=EPOCHS, verbose=2)
gru.fit(X_train, y_train, epochs=EPOCHS, verbose=2)
lstm.fit(X_train, y_train, epochs=EPOCHS, verbose=2)

Epoch 1/100


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


159/159 - 3s - 16ms/step - loss: 0.2456
Epoch 2/100
159/159 - 1s - 6ms/step - loss: 0.1166
Epoch 3/100
159/159 - 1s - 6ms/step - loss: 0.1097
Epoch 4/100
159/159 - 1s - 6ms/step - loss: 0.1085
Epoch 5/100
159/159 - 1s - 6ms/step - loss: 0.1082
Epoch 6/100
159/159 - 1s - 6ms/step - loss: 0.1084
Epoch 7/100
159/159 - 1s - 6ms/step - loss: 0.1083
Epoch 8/100
159/159 - 1s - 6ms/step - loss: 0.1082
Epoch 9/100
159/159 - 1s - 6ms/step - loss: 0.1074
Epoch 10/100
159/159 - 1s - 6ms/step - loss: 0.1080
Epoch 11/100
159/159 - 1s - 6ms/step - loss: 0.1068
Epoch 12/100
159/159 - 1s - 6ms/step - loss: 0.1072
Epoch 13/100
159/159 - 1s - 6ms/step - loss: 0.1070
Epoch 14/100
159/159 - 1s - 6ms/step - loss: 0.1061
Epoch 15/100
159/159 - 1s - 6ms/step - loss: 0.1065
Epoch 16/100
159/159 - 1s - 6ms/step - loss: 0.1066
Epoch 17/100
159/159 - 1s - 6ms/step - loss: 0.1067
Epoch 18/100
159/159 - 1s - 6ms/step - loss: 0.1068
Epoch 19/100
159/159 - 1s - 6ms/step - loss: 0.1065
Epoch 20/100
159/159 - 1s - 6ms/

In [35]:
rnn_preds = rnn.predict(X_test)
gru_preds = gru.predict(X_test)
lstm_preds = lstm.predict(X_test)

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


In [36]:
results = {
'RNN': {
'MAE': mean_absolute_error(y_test, rnn_preds),
'RMSE': np.sqrt(mean_squared_error(y_test, rnn_preds))
},
'GRU': {
'MAE': mean_absolute_error(y_test, gru_preds),
'RMSE': np.sqrt(mean_squared_error(y_test, gru_preds))
},
'LSTM': {
'MAE': mean_absolute_error(y_test, lstm_preds),
'RMSE': np.sqrt(mean_squared_error(y_test, lstm_preds))
}
}


results_df = pd.DataFrame(results).T
results_df

,MAE,RMSE
RNN,0.287054,0.372564
GRU,0.262858,0.347633
LSTM,0.266282,0.344729
